In [ ]:
# Reads all coding sequences from the LC300 genome and extracts gene id, protein id and aa sequence

In [87]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqUtils.ProtParam import ProteinAnalysis

In [88]:
import csv

In [89]:
# Read LC300 genome from genbank file
with open("LC300_genome.gb", 'r') as f:
    genome = SeqIO.read(f, 'gb')
    f.close()

In [110]:
# Save all coding sequences to cds
cds = []
for feature in genome.features:
    if feature.type == "CDS":
        cds.append(feature)
        
# Put gene id, protein id, and translation into array
array = [['Gene', 'Protein', 'Name', 'Protein_sequence']];
for i in range(len(cds)):   #range(len(features)):
    locus = str(cds[i].qualifiers["locus_tag"]).strip("[\']")
    protein = str(cds[i].qualifiers["protein_id"]).strip("[\']")
    seq = str(cds[i].qualifiers["translation"]).strip("[\']")
    name = str(cds[i].qualifiers["product"]).strip("[\']")
    array.append([locus, protein,name, seq])

In [91]:
print(len(cds))

2825


In [92]:
print(cds[15])

type: CDS
location: [18103:19840](+)
qualifiers:
    Key: codon_start, Value: ['1']
    Key: inference, Value: ['EXISTENCE: similar to AA sequence:RefSeq:WP_020754857.1']
    Key: locus_tag, Value: ['IB49_00115']
    Key: note, Value: ['Derived by automated computational analysis using gene prediction method: Protein Homology.']
    Key: product, Value: ['ABC transporter ATP-binding protein']
    Key: protein_id, Value: ['AKU25195.1']
    Key: transl_table, Value: ['11']
    Key: translation, Value: ['MGRVLIYLRPYGKWMALAWLFMLTELIIELWQPLLMGKIIDEGIMKQDVAAIFTWGAVMLGASLLAFASGIANSFAAAYVGQEYGFALRTALFAKIQSFSLARIEQLSPASLITRMTNDVTQVQNMLFMSLRIALRAPLLVVFGVAMAFVVHARLALVLAVAVPLSAVFLLWVVQKAAASFSAVQRALDRVNGVMRENLAGMRLIKAWMRKEYEEERFATANDALMERTMSVLRLVETITPVLLMVMNTAIVAVLLFGRHHIEDGTASAGQVVAVINYATRTTAALSMFTFITMAFSRARASAARLAELLEAPGERHGADAGGGPAVQRGEIRFEHVSFRYPDNDLDALSDVSFVIRPHETAAILGATGAGKSTLLQLIPRLYEPSGGRVLIDGVDVREFSAEQLRTAVRFVPQEVLLFSGTVADNLRFGKMTVTMEEIVQAARDAQIHETITRFPDGYDARIGQKGVNLSGGQKQRLSIARALVGQPRILLL

In [94]:
for i in range(len(array)):
    print(array[i])

['Gene', 'Protein', 'Name', 'Protein_sequence']
['IB49_00015', 'AKU25180.1', 'hypothetical protein', 'MKKKRFTVAEGETIAACLARMKQEGYRPVRRIEQPIFREVETNGETMVEPCGRIIEFEGVRDEP']
['IB49_00020', 'AKU25181.1', 'hypothetical protein', 'MMDEQESKRQFQDDLDQYRMDNVIHAPKHYVYQVGYEASSGNPTGGHRDAKKTREWPHS']
['IB49_00030', 'AKU25182.1', 'hypothetical protein', 'MIHHTWATRPTIKKVKCVHTNAEKYMVSNVLTPGKVYEVKNETDEFYFVIDNSGKVGGFYKDYFEEVK']
['IB49_00040', 'AKU25183.1', 'hypothetical protein', 'MHWLCPVFQQPNRQDAKERQHAAKPHSCAVRRQIGDFAEHDRTKRPHAKHQADDESGRQADMVRKQPVA']
['IB49_00045', 'AKU25184.1', 'membrane associated protein', 'MREDFRLPPHPVYVPVTLIRDGQLLADELAELGKTEQWLAAKLQKQGIASPKDVLIAEWLEGDGLFVQTYQPAERQRSTRRQTAPE']
['IB49_00050', 'AKU25185.1', 'GntR family transcriptional regulator', 'MFELDIRSRQPIYEQLIDKMKEMIVRELWQPHDQLPSVRTMAKQLMVNPNTIQKAYRELERDGWIYSIPGKGSFVAPRSKEPNIEAIAAVREQFVRLVKEARFLGVTNEQLWQWIREGEKEEGES']
['IB49_00055', 'AKU25186.1', 'ABC transporter', 'MIQLVDVTKMFDRFAAVKGANMMVPKGAIYGLLGPNGAGKTTLLKMMAGILRQDRGTIAVDGEDVW

In [105]:
# Some genes (9) in the model are not found as CDS. These where manually added to iGEL604 to cover gaps, 
# but correspond to genes listed as pseudogenes. For the model, these are manually extracted from the genbank file:

manualGeneList = ['IB49_16075','IB49_13335','IB49_00430','IB49_05560','IB49_04485','IB49_16565','IB49_00915','IB49_00960','IB49_1815']

In [101]:
# Searches for specific locus tag and hte matching coding sequence. Tested with the "missing" genes
for feature in genome.features:
    if feature.type == 'CDS' or feature.type == 'gene':
        if str(feature.qualifiers.get('locus_tag')).lstrip('[\'').rstrip('\']') in manualGeneList:
            if '+' in str(feature.location):
                coding_seq = genome.seq[feature.location.start:feature.location.end]
            else:
                coding_seq = genome.seq[feature.location.start:feature.location.end].reverse_complement()
            aa_seq = str(coding_seq.translate(table=11, to_stop=True))  # Goes only to first stop codon. Creates
            # potentially unrealistically small enzyme MW. Double check genome sequence?
            
            mw = ProteinAnalysis(aa_seq).molecular_weight()/1000
            print(feature)
            print("Coding sequence: %s" %(coding_seq))
            print("Protein sequence: %s" %(aa_seq))
            print("Protein MW: %.2f kDa" %(mw))

type: gene
location: [71659:73868](+)
qualifiers:
    Key: locus_tag, Value: ['IB49_00430']
    Key: note, Value: ['hydroperoxidase; disrupted; Derived by automated computational analysis using gene prediction method: Protein Homology.']
    Key: pseudo, Value: ['']

Coding sequence: ATGGAAAATCAAAATCGTCAGAATGCAGCCCAATGTCCGTTTCACGGAAGCGTAACGAATCAGTCTTCGAATCGAACGACGAACAAAGACTGGTGGCCGAACCAGCTGAACTTAAGCATTCTCCATCAACATGACGAAAAACGAATCCTCATGATGAAGAGTTCAACTATGCTGAGGAGTTTCAAAAACTAGACTATTGGGCGCTCAAAGAAGATTTGCGCAAACTGATGACAGAAAGCCAAGATTGGTGGCCGGCCGATTATGGCCATTACGGTCCGCTGTTCATCCGCATGGCGTGGCATTCGGCGGGAACGTACCGCATCGGCGACGGCCGCGGCGGCGGTTCGACCGGCACGCAGCGCTTTGCCCCGTTAAACAGCTGGCCGGATAACGCCAACTTGGATAAAGCGCGGCGGTTGCTGTGGCCGATCAAGAAAAAATACGGCAACAAAATTTCTTGGGCTGACTTGATGATTTTGGCCGGCAATGTCGCCATCGAATCGATGGGCGGCAAAACGATCGGCTTTGGCGGCGGCCGGGAAGATGTCTGGCATCCGGAAGAAGACATTTATTGGGCGCGGAAAAAGAGTGGCTCGCTTCGGAACGCTACAGCGGGGATCGCGAACTGGAAAATCCGCTTGCGGCTGTGCAGATGGGGCTGATCTATGTCAACCCGGAAGGCCCTGACGGCAAGCCGGATCCGAAAGCGGCAGCG

In [111]:
i = 1
for feature in genome.features:
    if feature.type == 'gene':
        locus = str(feature.qualifiers.get('locus_tag')).lstrip('[\'').rstrip('\']')
        if locus in manualGeneList:
            if '+' in str(feature.location):
                coding_seq = genome.seq[feature.location.start:feature.location.end]
            else:
                coding_seq = genome.seq[feature.location.start:feature.location.end].reverse_complement()
            aa_seq = str(coding_seq.translate(table=11, to_stop=True))  # Goes only to first stop codon. Creates
            protein = "Pseudoprot%s" %(i)
            name = "hypothetical protein"
            i+=1
            array.append([locus, protein,name, aa_seq])
            print(protein)


Pseudoprot1
Pseudoprot2
Pseudoprot3
Pseudoprot4
Pseudoprot5
Pseudoprot6
Pseudoprot7
Pseudoprot8


In [112]:
with open('gene_prot_aaseq.csv', 'w') as f:
    writer = csv.writer(f, delimiter=',')
    writer.writerows(array)
    f.close()

6283
